In [ ]:
import os
from datetime import datetime
import pandas as pd

# Создание папки для сохранения результатов
result_path = f'Результат/{datetime.now().strftime("%Y-%m-%d")}/'
if not os.path.exists(result_path):
    os.makedirs(result_path)

# Определение названий столбцов для данных из СБИС
sbis_names = [
    "Дата", "Номер", "Сумма", "Статус", "Примечание", "Комментарий", "Контрагент", "ИНН/КПП", "Организация", 
    "ИНН/КПП", "Тип документа", "Имя файла", "Дата", "Номер 1", "Сумма 1", "Сумма НДС", "Ответственный", 
    "Подразделение", "Код", "Дата", "Время", "Тип пакета", "Идентификатор пакета", "Запущено в обработку", 
    "Получено контрагентом", "Завершено", "Увеличение суммы", "НДC", "Уменьшение суммы", "НДС"
]

# Загрузка данных из СБИС
sbis_folder = "Входящие"
sbis_files = os.listdir(sbis_folder)

# Чтение всех CSV-файлов из папки СБИС и объединение их в один DataFrame
dfs = []
for file in sbis_files:
    if 'csv' not in file:
        continue
    df = pd.read_csv(os.path.join(sbis_folder, file), skiprows=1, sep=";", encoding="windows-1251", header=None)
    dfs.append(df)

sbis = pd.concat(dfs, ignore_index=True)
sbis.columns = sbis_names
sbis.columns = [c.replace(' ', '_') for c in sbis.columns]  # Замена пробелов в названиях столбцов на "_"

# Загрузка данных из папки "Аптеки"
apteka_folder = "Аптеки/csv/correct/"
apteka_files = os.listdir(apteka_folder)

# Обработка каждого файла из папки "Аптеки"
for file in apteka_files:
    if 'csv' not in file:
        continue

    # Чтение CSV-файла
    apteka = pd.read_csv(os.path.join(apteka_folder, file), sep=";", encoding="windows-1251")
    
    # Добавление новых столбцов
    apteka["Номер счет-фактуры"] = ""
    apteka["Сумма счет-фактуры"] = ""
    apteka["Дата счет-фактуры"] = ""
    apteka["Сравнение дат"] = ""

    # Типы документов для фильтрации
    docs = ["СчФктр", "УпдДоп", "УпдСчфДоп", "ЭДОНакл"]

    # Обработка каждой строки в DataFrame
    for i, row in apteka.iterrows():
        nakl = row["Номер накладной"]

        # Корректировка номера накладной для поставщика "ЕАПТЕКА"
        if 'ЕАПТЕКА' in row["Поставщик"]:
            nakl += "/15"

        # Поиск соответствующих записей в данных СБИС
        records = sbis[sbis.Номер.values == nakl]
        records = records[records.Тип_документа.isin(docs)]

        if records.empty:
            continue

        # Извлечение данных из первой найденной записи
        invoice = records.iloc[0]["Номер"]
        summ = records.iloc[0]["Сумма"]
        date = records.iloc[0]["Дата"][1]
        date = datetime.strptime(date, "%d.%m.%y").strftime("%d.%m.%Y")

        # Заполнение данных в DataFrame
        apteka.at[i, "Номер счет-фактуры"] = invoice
        apteka.at[i, "Сумма счет-фактуры"] = summ
        apteka.at[i, "Дата счет-фактуры"] = date
        apteka.at[i, "Сравнение дат"] = "" if (date == apteka.at[i, 'Дата накладной']) else "Не совпадает!"

    # Выбор и упорядочивание столбцов для сохранения
    apteka_columns = [
        '№ п/п', 'Штрих-код партии', 'Наименование товара', 'Поставщик', 'Дата приходного документа', 
        'Номер приходного документа', 'Дата накладной', 'Номер накладной', 'Номер счет-фактуры', 
        'Сумма счет-фактуры', 'Кол-во', 'Сумма в закупочных ценах без НДС', 'Ставка НДС поставщика', 
        'Сумма НДС', 'Сумма в закупочных ценах с НДС', 'Дата счет-фактуры', 'Сравнение дат'
    ]

    apteka = apteka[apteka_columns]

    # Сохранение результата в Excel
    apteka.to_excel(f"{result_path}{file.split('.csv')[0]} - результат.xlsx", index=False)
    print(f'{file} Обработан!')

C:\Users\r.barannikov\AppData\Local\Temp\ipykernel_23152\3990792480.py:74: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  date = records.iloc[0]["Дата"][1]


366.csv Обработан!


C:\Users\r.barannikov\AppData\Local\Temp\ipykernel_23152\3990792480.py:74: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  date = records.iloc[0]["Дата"][1]


А123.csv Обработан!
